In [1]:
%load_ext autoreload
%autoreload 2

import os
import zarr
import dask.array as da
import napari
import sys
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append('..')
from tracks_interactions.db.db_model import Base, CellDB, TrackDB
import tracks_interactions.graph.family_graph as fg
from tracks_interactions.widget.event_handler import EventHandler

In [2]:
os.environ['NAPARI_PERFMON'] = '0'

In [3]:
labels_zarr_path = r'D:\kasia\tracking\E6_exp\code\tests\example_track_labels.zarr'
labels = da.from_zarr(labels_zarr_path)
labels.shape

(241, 8396, 8401)

In [5]:
labels = np.array(labels).astype('int32')

In [7]:
# display the labels
viewer = napari.Viewer()
labels_layer = viewer.add_labels(labels,name='Labels')

In [8]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20_dev\cells_database_2tables - Copy.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

In [9]:
# create a plot widget
t_max = viewer.dims.range[0][1]
plot_widget = fg.build_lineage_widget(t_max)

# add lineage graph
viewer.window.add_dock_widget(plot_widget,area='bottom',name='family')

In [11]:
from napari import Viewer

def test_func(viewer: Viewer):
    print('just testing')

In [10]:
# connect lineage graph update
event_handler = EventHandler(viewer,plot_widget,session)
labels_layer.events.selected_label.connect(event_handler.lineage_update)

<bound method EventHandler.lineage_update of <tracks_interactions.widget.event_handler.EventHandler object at 0x000002E9E4CE87F0>>

In [12]:
labels_layer.events.labels_update.connect(test_func)

<function __main__.test_func(viewer: napari.viewer.Viewer)>

just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing
just testing

In [8]:
from magicgui import magicgui
from qtpy.QtGui import QIcon
from napari import Viewer
from tracks_interactions.widget.track_operations import cut_track_function, merge_track_function
from functools import partial

In [9]:
######################################################################
# add cut tracks button
cut_track_function_partial = partial(cut_track_function, session=session)

cut_track_btn = magicgui(cut_track_function_partial)
path_to_some_icon = r'../tracks_interactions/icons/icons8-scissors-50.png'

icon = QIcon(path_to_some_icon)
cut_track_btn.call_button.native.setIcon(icon)
cut_track_btn.call_button.native.setText(None)

viewer.window.add_dock_widget(cut_track_btn,area='right',name='cut_track')
cut_track_btn.call_button.native.resize(40,40)

In [10]:
@magicgui(
        t1={"widget_type": "SpinBox", "max": 1000000},
        call_button = 'Merge'
        )  

def merge_track_function_partial(viewer: Viewer, t1: int):
    return merge_track_function( viewer, session=session, t1=t1)

viewer.window.add_dock_widget(merge_track_function_partial,area='right',name='t1')

In [11]:
session.query(TrackDB).filter_by(track_id=75034).first()

Track 75034 from 118 to 128